In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [13]:
dataset_path = '/content/drive/MyDrive/Nhung/TRAINMODEL/Data'

In [14]:
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
IMAGE_CHANNELS = 1
NUM_CLASSES = 2

In [15]:
model = Sequential([
    Flatten(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)),
    Dense(8, activation='relu'),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

In [16]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [19]:
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=32,
    color_mode='grayscale',
    class_mode='sparse',
    subset='training'
)

Found 64 images belonging to 2 classes.


In [20]:
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=32,
    color_mode='grayscale',
    class_mode='sparse',
    subset='validation'
)

Found 16 images belonging to 2 classes.


In [21]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
2/2 [==============================] - 23s 21s/step - loss: 0.4185 - accuracy: 1.0000 - val_loss: 0.2214 - val_accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 159ms/step - loss: 0.1916 - accuracy: 1.0000 - val_loss: 0.0903 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 246ms/step - loss: 0.0774 - accuracy: 1.0000 - val_loss: 0.0335 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 160ms/step - loss: 0.0291 - accuracy: 1.0000 - val_loss: 0.0124 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 165ms/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 175ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 152ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 8/10
2/2 [=====

In [30]:
from tensorflow.keras.preprocessing import image
image_path = "/content/drive/MyDrive/Nhung/TRAINMODEL/Data/1/User-1-14.jpg"

test_image = image.load_img(image_path, target_size=(32, 32), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0

# Dự đoán
result = model.predict(test_image)
class_id = np.argmax(result)

# Hiển thị kết quả
print("Class Predicted:", class_id)

1/1 [==============================] - 0s 17ms/step
Class Predicted: 0


In [24]:
model.save('model2.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/model2.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model32.tflite', 'wb') as f:
    f.write(tflite_model)

In [26]:

# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [27]:
with open("c_model_name" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "c_model_name2"))